In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,335 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,428 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [581 kB]
Get:12 https://cloud.r-pr

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UDF").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType


In [4]:
from google.colab import files

uploaded = files.upload()

Saving NFL_COMBINE.csv to NFL_COMBINE.csv


In [5]:
import pandas as pd
import io
  
df = pd.read_csv('NFL_COMBINE.csv', header= 0,
                        encoding= 'unicode_escape')
df.head()

,Year,ACC,Big_Ten,Big_12,Pac_12,SEC,Other,Height_IN,Weight_LBS,Wonderlic,40_Yard,Bench_Press,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started
0,2020.0,0.0,0.0,0.0,0.0,0.0,1.0,75.13,229.0,NaN,4.69,NaN,35.0,125.0,4.51,7.33,No
1,2020.0,0.0,0.0,0.0,0.0,0.0,0.0,75.63,221.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2,2020.0,0.0,0.0,0.0,0.0,0.0,1.0,76.13,224.0,NaN,5.00,NaN,28.0,109.0,4.37,7.13,No
3,2020.0,0.0,0.0,0.0,1.0,1.0,0.0,77.88,231.0,NaN,4.89,NaN,27.5,110.0,4.75,7.50,No
4,2020.0,0.0,0.0,0.0,0.0,0.0,0.0,73.88,219.0,NaN,5.01,NaN,30.0,111.0,4.51,7.27,No


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           603 non-null    float64
 1   ACC            603 non-null    float64
 2   Big_Ten        603 non-null    float64
 3   Big_12         603 non-null    float64
 4   Pac_12         603 non-null    float64
 5   SEC            635 non-null    float64
 6   Other          635 non-null    float64
 7   Height_IN      603 non-null    float64
 8   Weight_LBS     603 non-null    float64
 9   Wonderlic      208 non-null    float64
 10  40_Yard        536 non-null    float64
 11  Bench_Press    34 non-null     float64
 12  Vert_Leap_IN   508 non-null    float64
 13  Broad_Jump_IN  509 non-null    float64
 14  Shuttle        491 non-null    float64
 15  3Cone          337 non-null    float64
 16  Started        603 non-null    object 
dtypes: float64(16), object(1)
memory usage: 93.8+ KB


In [7]:
df.drop(columns=["Wonderlic","Bench_Press"],axis=1, inplace=True)

In [8]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 414
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           328 non-null    float64
 1   ACC            328 non-null    float64
 2   Big_Ten        328 non-null    float64
 3   Big_12         328 non-null    float64
 4   Pac_12         328 non-null    float64
 5   SEC            328 non-null    float64
 6   Other          328 non-null    float64
 7   Height_IN      328 non-null    float64
 8   Weight_LBS     328 non-null    float64
 9   40_Yard        328 non-null    float64
 10  Vert_Leap_IN   328 non-null    float64
 11  Broad_Jump_IN  328 non-null    float64
 12  Shuttle        328 non-null    float64
 13  3Cone          328 non-null    float64
 14  Started        328 non-null    object 
dtypes: float64(14), object(1)
memory usage: 41.0+ KB


In [10]:
df.drop(columns=["Year"],axis=1, inplace=True)

In [11]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 414
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ACC            328 non-null    float64
 1   Big_Ten        328 non-null    float64
 2   Big_12         328 non-null    float64
 3   Pac_12         328 non-null    float64
 4   SEC            328 non-null    float64
 5   Other          328 non-null    float64
 6   Height_IN      328 non-null    float64
 7   Weight_LBS     328 non-null    float64
 8   40_Yard        328 non-null    float64
 9   Vert_Leap_IN   328 non-null    float64
 10  Broad_Jump_IN  328 non-null    float64
 11  Shuttle        328 non-null    float64
 12  3Cone          328 non-null    float64
 13  Started        328 non-null    object 
dtypes: float64(13), object(1)
memory usage: 38.4+ KB


In [12]:
X = pd.get_dummies(df)
X

,ACC,Big_Ten,Big_12,Pac_12,SEC,Other,Height_IN,Weight_LBS,40_Yard,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started_No,Started_Yes
0,0.0,0.0,0.0,0.0,0.0,1.0,75.13,229.0,4.69,35.0,125.0,4.51,7.33,1,0
2,0.0,0.0,0.0,0.0,0.0,1.0,76.13,224.0,5.00,28.0,109.0,4.37,7.13,1,0
3,0.0,0.0,0.0,1.0,1.0,0.0,77.88,231.0,4.89,27.5,110.0,4.75,7.50,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,73.88,219.0,5.01,30.0,111.0,4.51,7.27,1,0
6,0.0,0.0,0.0,1.0,0.0,1.0,78.25,236.0,4.68,35.5,123.0,4.46,7.06,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,0.0,0.0,0.0,0.0,0.0,1.0,78.10,224.0,5.00,31.5,112.0,4.39,7.54,1,0
411,0.0,0.0,0.0,1.0,0.0,0.0,74.30,195.0,4.96,30.0,111.0,4.29,7.30,0,1
412,0.0,1.0,0.0,0.0,0.0,0.0,76.40,221.0,4.95,31.0,109.0,4.47,7.97,0,1
413,0.0,0.0,0.0,0.0,0.0,0.0,72.40,208.0,4.96,30.0,104.0,4.51,8.16,1,0


In [13]:
X.drop(columns=["Started_No"],axis=1, inplace=True)

In [14]:
X.head()

,ACC,Big_Ten,Big_12,Pac_12,SEC,Other,Height_IN,Weight_LBS,40_Yard,Vert_Leap_IN,Broad_Jump_IN,Shuttle,3Cone,Started_Yes
0,0.0,0.0,0.0,0.0,0.0,1.0,75.13,229.0,4.69,35.0,125.0,4.51,7.33,0
2,0.0,0.0,0.0,0.0,0.0,1.0,76.13,224.0,5.00,28.0,109.0,4.37,7.13,0
3,0.0,0.0,0.0,1.0,1.0,0.0,77.88,231.0,4.89,27.5,110.0,4.75,7.50,0
4,0.0,0.0,0.0,0.0,0.0,0.0,73.88,219.0,5.01,30.0,111.0,4.51,7.27,0
6,0.0,0.0,0.0,1.0,0.0,1.0,78.25,236.0,4.68,35.5,123.0,4.46,7.06,1


In [15]:
y=X["Started_Yes"]
X.drop(columns=["Started_Yes"],axis=1, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [18]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.6341463414634146
Testing Data Score: 0.6463414634146342


In [21]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
classifier.fit(X_train, y_train)
print(classifier)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
def getTestData(ACC, Big_Ten, Big_12,	Pac_12, SEC, Other,	Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone):
  testData = [ACC, Big_Ten, Big_12,Pac_12, SEC, Other,	Height_IN,	Weight_LBS,	40_Yard,	Vert_Leap_IN,	Broad_Jump_IN,	Shuttle,	3Cone]
  return testData

SyntaxError: ignored

In [28]:
new_data=[0.0,	0.0, 0.0,	0.0,	0.0,	1.0,	75.13,	229.0,		4.69	,	35.0,	125.0,	4.51,	7.33]

In [33]:
import numpy as np
testData=np.reshape(new_data,(1,-1))

In [35]:
predictions = classifier.predict(testData)
print("Players are either 0 (not a starter) or 1 (stared)")
print(f"The player was classified as: {predictions}")

Players are either 0 (not a starter) or 1 (stared)
The player was classified as: [0]
